<a id='report-production'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Parameter Reports</span> </p>

For demonstration purposes we import here data for some *Group of Insurance Contract* (GIC) and *Group of Reinsurance Contract* (GRIC) - the import is triggered in the [Set up data and configuration](#set-up-data-and-configuration) section.
<br> The imported data set consists of cash flows, actuals, and parameters.
<br> Input files can be found in the **File** directory. You are invited to change them or upload your own or add new data in the [CloseImportTemplate](../Import/CloseImportTemplate) notebook. 

In this notebook we show the parameters (provided to the calculation engine as inputs) used to performe the calculation of the reports shown in [Reports](Reports) notebook.

# Set up data and configuration

Choose to run the Reports notebook either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!eval-notebook "../Import/CloseImportTemplate"

In [0]:
Workspace.InitializeFrom(DataSource);

# Args

Define the reporting node, year, month, and scenario for which the parameter reports should be loaded. 

In [0]:
var reportinNode = "CH";
var year = 2021;
var month = 3;
var scenario = (string)default;
var args = new ImportArgs(reportinNode, year, month, default, scenario, default);

# Retrieve data 

The parameters used for calculation of the provided period are here retrieved through queries.
The queries and the data model of the reported objects can be foud in the [ParameterReportQueries](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/Report/ParameterReportsQueries#queries)

In [0]:
var dataNodeData = await Workspace.GetDataNodeDataReportParametersAsync(args);

In [0]:
var dataNodeStates = await Workspace.GetDataNodeStateReportParametersAsync(args);

In [0]:
var allYieldCurves = await Workspace.GetYieldCurveReportParametersAsync(args);

In [0]:
var singleDataNodeParameters = await Workspace.GetSingleDataNodeReportParametersAsync(args);

In [0]:
var interDataNodeParameters = await Workspace.GetInterDataNodeParametersAsync(args);

In [0]:
var currentPartnerRatings = await Workspace.GetCurrentPartnerRatingsReportParametersAsync(args);

In [0]:
var lockedPartnerRatings = await Workspace.GetLockedInPartnerRatingsReportParametersAsync(args);

In [0]:
var currentPartnerDefaultRates = await Workspace.GetCurrentCreditDefaultRatesReportParametersAsync(args);

In [0]:
var lockedPartnerDefaultRates = await Workspace.GetLockedInCreditDefaultRatesReportParametersAsync(args);

# Reports

## Data node
Properties of the Group of Contracts and corresponding Portfolios are merged to provide a full description of the [Data Node](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructureDataStructure#data-node)

In [0]:
await Report.ForObjects(dataNodeData)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.Portfolio).GroupRowsBy(x => x.DataNode)
            .ToTable()
            .ExecuteAsync()

## Data node state

Current and previous period [data node state](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#data-node-state).

In [0]:
await Report.ForObjects(dataNodeStates)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()
            

## Yield curve
[Yield Curve](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#yield-curve) used for locked-in discounting and current rating discouning (curret period and previous period) are shown.

In [0]:
await Report.ForObjects(allYieldCurves)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.YieldCurveType).GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

## Single data node parameter
[Single data node parameters](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#data-node-parameters) for current and previous period. 

In [0]:
await Report.ForObjects(singleDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

## Inter data node parameter
[Inter data node parameters](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#data-node-parameters) for current and previous period. 

In [0]:
await Report.ForObjects(interDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract).GroupRowsBy(x => x.LinkedDataNode)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

## Partner ratings
[Partner ratings](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#partner-rating) for current and previous period.

In [0]:
await Report.ForObjects(currentPartnerRatings)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.Partner)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

Locked in [partner ratings](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#partner-rating).

In [0]:
await Report.ForObjects(lockedPartnerRatings)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.Partner)
            .GroupColumnsBy(x => x.PartnerRatingType).GroupColumnsBy(x => "Initial Year: " + x.InitialYear.ToString())
            .ToTable()
            .ExecuteAsync()

## Partner default rates
[Partner default rates](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#credit-default-rate) for current and previous period.

In [0]:
await Report.ForObjects(currentPartnerDefaultRates)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.CreditRiskRating)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

Locked in [partner default rates](https://portal.systemorph.cloud/project/ifrs17/env/v1.3.1/DataModel/DataStructure#credit-default-rate).

In [0]:
await Report.ForObjects(lockedPartnerDefaultRates)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.CreditRiskRating)
            .GroupColumnsBy(x => x.CreditDefaultRatesType).GroupColumnsBy(x => "Initial Year: " + x.InitialYear)
            .ToTable()
            .ExecuteAsync()